Import Libraries


In [ ]:
import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
from spacy.matcher import Matcher 
from spacy.tokens import Span 
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm

Check for Sample Data

In [ ]:
doc = nlp("Phytoplasmas are insect-vectored bacteria that cause disease in a wide range of plant species.")

In [ ]:
for tok in doc:
  print(tok.text, "...", tok.dep_)

Phytoplasmas ... nsubj
are ... ROOT
insect ... npadvmod
- ... punct
vectored ... amod
bacteria ... attr
that ... nsubj
cause ... relcl
disease ... dobj
in ... prep
a ... det
wide ... amod
range ... pobj
of ... prep
plant ... compound
species ... pobj
. ... punct


In [ ]:
pd.set_option('display.max_colwidth', 200)
#%matplotlib inline


Read Dataset

In [ ]:
data = pd.read_csv("X.csv")
print("\n\nData")
print(data.shape)



Data
(62197, 1)


In [ ]:
print(data)

                                                                                                                                                                                                      sentence
0                                                                                                                Phytoplasmas are insect-vectored bacteria that cause disease in a wide range of plant species
1       The increasing availability of molecular DNA analyses expertise and additional methods in recent years has led to a proliferation of discoveries of phytoplasma-plant host associations and in the ...
2       The widespread use of common names based on the diseases with which they are associated as well as separate phenetic and taxonomic systems for classifying phytoplasmas based on variation at the 1...
3       We explore this issue and related trends through a focus on Australian pathosystems providing the first comprehensive compilation of information for this continent 

Function to create entities/Keywords

In [ ]:
def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    
  prv_tok_text = ""   

  prefix = ""
  modifier = ""

  
  
  for tok in nlp(sent):
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]

Extraction of Entities and Entity Pairs

In [ ]:
entities = get_entities("the temple is set on fire")
print(entities)

['temple', 'fire']


In [ ]:

entity_pairs = []

for i in tqdm(data["sentence"]):
  entity_pairs.append(get_entities(i))
  
print(entity_pairs)

100%|██████████| 62197/62197 [11:19<00:00, 91.49it/s]

[['vectored  that', 'wide plant species'], ['molecular  analyses', 'taxonomic plant host phytoplasmas'], ['common  they', 'encoding  literature'], ['We', 'comprehensive phytoplasmas vectors'], ['this', 'additional pathogen groups'], ['unrecognized  groups', 'well 16Sr groups'], ['Wide host plant ranges', 'crop  some'], ['putative  vectors', 'especially  Australia'], ['', 'studied plant pathogens'], ['unsaturated sesquiterpene that', 'inflammatory skin properties'], ['natural  habitat', 'threat'], ['microbial  production', 'renewable -)-alpha feedstock'], ['Here  we', 'natural vegetable oils'], ['German Matricaria titer', 'biosynthetic mevalonate pathway'], ['mg  L', 'bisabolol  diphosphate'], ['Combinatorial  expression', 'engineered L bioreactor E'], ['', '-)-alpha bisabolol -)-alpha synthase'], ['batch  fermentation', 'bisabolol culture 9'], ['', ''], ['downstream extraction process', 'high yield recovery'], ['', ''], ['coli  strains', 'large -)-alpha scale'], ['universal  feature', 